In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessing-classes/preprocessing_utils.py
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/pipeline/preprocessing_pipeline_final (1).pkl


In [3]:
!pip install dagshub mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 62.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 89.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

In [6]:
import dagshub
dagshub.init(repo_owner='ekvirika', repo_name='HousePrices', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=1175a2b0-5c7e-48b8-a701-e34a9197e788&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=b75f2c86fd2c850f798a13e0682ffff9c7bd90e4a671ac43291b5935a41f06e7




Accessing as ekvirika

Initialized MLflow to track repo "ekvirika/HousePrices"

Repository ekvirika/HousePrices initialized!

# Import pipeline

In [5]:
import sys
sys.path.append("/kaggle/input/preprocessing-classes")

from preprocessing_utils import * 

In [8]:
import joblib

# Path to the uploaded pipeline
pipeline_path = "/kaggle/input/pipeline/preprocessing_pipeline_final (1).pkl"

# Load the pipeline
pipeline = joblib.load(pipeline_path)
print(type(pipeline))
print(pipeline)


<class 'sklearn.pipeline.Pipeline'>
Pipeline(steps=[('data_cleaner',
                 DataCleaner(null_threshold=0.9, target_column='SalePrice')),
                ('null_handler', NullHandler(null_threshold=0.9)),
                ('one_hot_encoder', SelectiveOneHotEncoder(num_unique=3)),
                ('woe_encoder', WoECategoricalEncoder()),
                ('correlation_filter',
                 CorrelationFilter(target_col='SalePrice'))])


# Transform testset

In [28]:
X_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
original_ids = X_test['Id'].copy()  # Store original IDs before dropping

In [29]:
X_test.drop(columns='Id', inplace=True)
X_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [24]:
X_test_processed = pipeline.transform(X_test)


Transforming dataset with DataCleaner...
Dropping columns: ['RoofMatl', 'KitchenAbvGr', 'MiscFeature', 'Street', 'PoolArea', 'GarageCond', 'Utilities', 'Condition2', 'LowQualFinSF', '3SsnPorch', 'Heating', 'MiscVal', 'PoolQC', 'GarageQual', 'Alley']
Shape after cleaning: (1459, 65)
Transforming dataset with NullHandler...
Filling column 'MSSubClass' with: 50.0
Filling column 'LotFrontage' with: 70.0
Filling column 'LotArea' with: 9600.0
Filling column 'OverallQual' with: 6.0
Filling column 'OverallCond' with: 5.0
Filling column 'YearBuilt' with: 1972.0
Filling column 'YearRemodAdd' with: 1994.0
Filling column 'MasVnrArea' with: 0.0
Filling column 'BsmtFinSF1' with: 384.5
Filling column 'BsmtFinSF2' with: 0.0
Filling column 'BsmtUnfSF' with: 480.0
Filling column 'TotalBsmtSF' with: 997.5
Filling column '1stFlrSF' with: 1095.0
Filling column '2ndFlrSF' with: 0.0
Filling column 'GrLivArea' with: 1473.0
Filling column 'BsmtFullBath' with: 0.0
Filling column 'BsmtHalfBath' with: 0.0
Filling

In [25]:
X_test_processed.shape

(1459, 64)

# Load Model

In [45]:
import mlflow
logged_model = 'runs:/7029fa464d80457a8948ceeb5a505c12/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

2025/04/11 12:30:03 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - cloudpickle (current: 3.1.1, required: cloudpickle==3.1.0)
 - psutil (current: 7.0.0, required: psutil==5.9.5)
 - scipy (current: 1.15.2, required: scipy==1.13.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/04/11 12:30:03 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.10.12`, differs from the version of Python that is currently running, `Python 3.11.11`, and may be incompatible


In [46]:
# Make predictions
predictions = model.predict(X_test_processed)
pd.DataFrame({"Id": X_test.index, "SalePrice": predictions}).to_csv("predictions.csv", index=False)

In [47]:
submission_df = pd.DataFrame({
    "Id": original_ids,  # Use the original IDs, not the indexes
    "SalePrice": predictions
})

In [48]:
submission_df.to_csv("predictions.csv", index=False)

# Verify submission format
print(f"Submission shape: {submission_df.shape}")
print(f"ID range: {submission_df['Id'].min()} to {submission_df['Id'].max()}")

Submission shape: (1459, 2)
ID range: 1461 to 2919
